In [1]:
def read_dataframe(filename):
    import pandas as pd
    df= pd.read_parquet(filename)
    df['duration']=df['tpep_dropoff_datetime'] -df['tpep_pickup_datetime']
    df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1, inplace=True)
    df['duration']=df['duration'].dt.total_seconds()/60
    df=df[(df['duration'] >= 1) & (df['duration'] <= 60)]
    categorical_columns=['PULocationID' , 'DOLocationID']
    for column in categorical_columns:
        df[column] = df[column].astype('category')
    return df

In [2]:
df_train  = read_dataframe('data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('data/yellow_tripdata_2023-02.parquet')

In [3]:
len(df_val.columns)

18

In [4]:
import pandas as pd
df=pd.read_parquet('data/yellow_tripdata_2023-01.parquet')
df['duration']=df['tpep_dropoff_datetime'] -df['tpep_pickup_datetime']
df['duration']=df['duration'].dt.total_seconds()/60
df['duration'].describe()

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration, dtype: float64

In [5]:
total_records = len(df)
df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]
filtered_records = len(df)
fraction_remaining = filtered_records / total_records
print("Fraction of records remaining after removing outliers:", fraction_remaining)

Fraction of records remaining after removing outliers: 0.9812202822125979


In [6]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333


In [7]:
from sklearn.feature_extraction import DictVectorizer
train_df=df[['PULocationID', 'DOLocationID']]
records = train_df.to_dict(orient='records')
dv = DictVectorizer(sparse=False)
X = dv.fit_transform(records)
print("Dimensionality (number of columns):", X.shape[1])

Dimensionality (number of columns): 2


In [8]:
from sklearn.feature_extraction import DictVectorizer
categorical_columns=['PULocationID' , 'DOLocationID']
target = 'duration'
y_train =df_train[target].values
y_val =df_val[target].values

dv= DictVectorizer()
train_dict = df_train[categorical_columns ].to_dict(orient='records')
X_train=dv.fit_transform(train_dict)

val_dicts = df_val[categorical_columns ].to_dict(orient='records')
X_val =dv.transform(val_dicts)

In [9]:
X_train

<3009173x2 sparse matrix of type '<class 'numpy.float64'>'
	with 6018346 stored elements in Compressed Sparse Row format>

In [10]:
# Print the number of columns in X_train
print(X_train.shape[1])


2


In [11]:
X_val

<2855951x2 sparse matrix of type '<class 'numpy.float64'>'
	with 5711902 stored elements in Compressed Sparse Row format>

In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Linear Regression
lr = LinearRegression()
lr.fit(X_train, y_train)
lr_predictions = lr.predict(X_train)
lr_r2 = r2_score(y_train, lr_predictions)



In [25]:
lr_mse = mean_squared_error(y_train, lr_predictions, squared=False)

/home/codespace/anaconda3/envs/jupiter_env/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [26]:
print("Linear Regression - R^2:", lr_r2, "MSE:", lr_mse)

Linear Regression - R^2: 0.020838523768736184 MSE: 9.835277713631873


In [27]:
# Decision Tree Regressor
tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)
tree_predictions = tree.predict(X_train)
tree_r2 = r2_score(y_train, tree_predictions)




Decision Tree - R^2: 0.7442275638909809 MSE: 25.26810519621476


In [28]:
tree_mse = mean_squared_error(y_train, tree_predictions, squared = False)

/home/codespace/anaconda3/envs/jupiter_env/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [29]:
# Print results

print("Decision Tree - R^2:", tree_r2, "MSE:", tree_mse)

Decision Tree - R^2: 0.7442275638909809 MSE: 5.0267390220912365


In [16]:
X_val

<2855951x2 sparse matrix of type '<class 'numpy.float64'>'
	with 5711902 stored elements in Compressed Sparse Row format>

In [30]:
y_pred=lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

/home/codespace/anaconda3/envs/jupiter_env/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


9.963607595829973

In [18]:
import pickle

In [19]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [20]:
## pytorck

In [ ]:
pip install torch

In [21]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# Assume train_dict and df[target] are already defined and loaded as shown in your example
X_train = dv.fit_transform(train_dict).toarray()
X_val = dv.fit_transform(X_val).toarray()
y_train = df_train[target].values
y_val = df_val[target].values

# Convert arrays to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)


X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)


# Create DataLoader for batch processing
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

val_data = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)


ModuleNotFoundError: No module named 'torch'

In [ ]:
import torch.nn as nn

class TaxiTripModel(nn.Module):
    def __init__(self):
        super(TaxiTripModel, self).__init__()
        self.layer1 = nn.Linear(X_train.shape[1], 128)
        self.layer2 = nn.Linear(128, 64)
        self.layer3 = nn.Linear(64, 32)
        self.output = nn.Linear(32, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.relu(self.layer3(x))
        x = self.output(x)
        return x

model = TaxiTripModel()


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

def train_model(model, train_loader, val_loader, loss_fn, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        for X_batch, y_batch in train_loader:
            preds = model(X_batch)
            loss = loss_fn(preds, y_batch.unsqueeze(1))
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        model.eval()
        with torch.no_grad():
            val_loss = sum(loss_fn(model(X_val), y_val.unsqueeze(1)) for X_val, y_val in val_loader) / len(val_loader)
        
        print(f'Epoch {epoch+1}, Loss: {loss.item()}, Validation Loss: {val_loss.item()}')

train_model(model, train_loader, val_loader, loss_fn, optimizer, num_epochs=10)


In [ ]:
with open('models/TaxiTripModel.bin', 'wb') as f_out:
    pickle.dump((dv, model), f_out)